# Iterative Closest Point

In this subsection, we will code the Iterative Closest Point algorithm to find the alignment between two point clouds without known correspondences. The point cloud that you will be using is the same as the one that you used in Assignment 1.

## 1: Procrustes alignment

1. Write a function that takes two point clouds as input wherein the corresponding points between the two point clouds are located at the same index and returns the transformation matrix between them.
2. Use the bunny point cloud and perform the procrustes alignment between the two bunnies. Compute the absolute alignment error after aligning the two bunnies.
3. Make sure your code is modular as we will use this function in the next sub-part.
4. Prove mathematically why the Procrustes alignment gives the best aligning transform between point clouds with known correspondences.


In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2022-01-09 01:35:47,676 - utils - NumExpr defaulting to 8 threads.


In [2]:
# Answer 1
def procrustes_alignment(pts1, pts2):
    # ptCloud1, ptCloud2 are of size 3 x n

    dim, N = pts1.shape
    centre1 = np.mean(pts1, axis = 1)[..., None]
    centre2 = np.mean(pts2, axis = 1)[..., None]
    
    pts1_centred = pts1 - centre1
    pts2_centred = pts2 - centre2

    S = pts1_centred @ pts2_centred.T
    U, D, Vt = np.linalg.svd(S)

    R = U @ Vt

    #C = np.eye(3)
    #C[2][2] = np.linalg.det(U @ Vt)
    #R = U @ C @ Vt

    if np.linalg.det(R) < 0:
        Vt[2, :] *= -1
        R = U @ Vt

    t = centre1 - R @ centre2

    T = np.eye(4)
    T[:3, :3], T[:3, 3] = R, t[:, 0]

    return [T, R, t]
    

In [3]:
def getRotationMatrix(alpha,beta,gama):

    R = np.zeros((3,3))
    R[0,0] = math.cos(alpha)*math.cos(beta)
    R[0,1] = math.cos(alpha)*math.sin(beta)*math.sin(gama) - math.sin(alpha)*math.cos(gama) 
    R[0,2] = math.cos(alpha)*math.sin(beta)*math.cos(gama) + math.sin(alpha)*math.sin(gama)
    R[1,0] = math.sin(alpha)*math.cos(beta) 
    R[1,1] = math.sin(alpha)*math.sin(beta)*math.sin(gama) + math.cos(alpha)*math.cos(gama)
    R[1,2] = math.sin(alpha)*math.sin(beta)*math.cos(gama) - math.cos(alpha)* math.sin(gama)
    R[2,0] = -1*math.sin(beta)
    R[2,1] = math.cos(beta)*math.sin(gama)
    R[2,2] = math.cos(beta)*math.cos(gama)

    return R

In [4]:
def visualiseBunnies(bunny, T):
    #Color the bunny red
    bunny.paint_uniform_color([1,0,0])

    #get the coordinate axes
    mesh_frame =  o3d.geometry.TriangleMesh.create_coordinate_frame(size=.1,origin=[0,0,0])

    #Get the transformed bunny
    bunny_transform = copy.deepcopy(bunny).transform(T)

    #Get the transformed coordinate axes
    mesh_frame_transform = copy.deepcopy(mesh_frame).transform(T)

    #Color the transformed bunny blue
    bunny_transform.paint_uniform_color([0, 0, 1])

    #Visualize all of them together
    o3d.visualization.draw_geometries([bunny, mesh_frame, bunny_transform, mesh_frame_transform])


In [5]:
#Calculating absolute alignment error as MSE

def absoluteError(pts1, pts2):
    error = np.mean(np.mean(np.square(pts1 - pts2)))
    return error

In [6]:
def getTransformedPoints(pts, R, t):
    #pts is a 3 x N matrix
    #print(" pts shape:", pts.shape, " R:", R.shape, "t shape:", t.shape, "\n")
    ptsTransformed = R @ pts
    for i in range(ptsTransformed.shape[1]):
        ptsTransformed[:, i] += t
    return ptsTransformed

In [7]:

#Visulaize initial configuration:
bunnyPath = "bunny.ply"
bunny = o3d.io.read_point_cloud(bunnyPath)
print(bunny)

#Get the position matrix
bunnyPts = np.asarray(bunny.points).T
#print(bunnyPts.T)


#Get the transformation
T_12 = np.eye(4)
R_12 = getRotationMatrix(0, 0, np.pi/2)
t_12 = np.array([0, 0, 0.5])
T_12[:3, :3], T_12[:3, 3] = R_12, t_12

bunnyPts2 = getTransformedPoints(bunnyPts, R_12, t_12)

#Get the transformed bunny
bunny_transform = copy.deepcopy(bunny).transform(T_12)

#Get the position matrix
bunnyTransformedPts = np.asarray(bunny_transform.points).T
#print(bunnyTransformedPts.T)

#visualiseBunnies(bunny, T_12)
[T, R, t] = procrustes_alignment(bunnyPts, bunnyTransformedPts)

print(T_12, "from fn:\n", T, " inv:\n", np.linalg.inv(T))

visualiseBunnies(bunny, T)
visualiseBunnies(bunny, T_12 @ T)

print("Is the following matrix should be I (4x4):\n", T_12 @ T)

bunnyPts2Transformed = R @ bunnyPts2 + t
print("The absolute error is:\n", absoluteError(bunnyPts, bunnyPts2Transformed))

PointCloud with 35947 points.
[[ 1.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00]
 [ 0.000000e+00  6.123234e-17 -1.000000e+00  0.000000e+00]
 [-0.000000e+00  1.000000e+00  6.123234e-17  5.000000e-01]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]] from fn:
 [[ 1.00000000e+00 -1.94885852e-16 -1.09017582e-16  5.55111512e-17]
 [ 1.14711926e-16 -6.39496469e-17  1.00000000e+00 -5.00000000e-01]
 [-1.50011672e-16 -1.00000000e+00  8.02068255e-17 -4.68375339e-17]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]  inv:
 [[ 1.00000000e+00  1.09017582e-16 -1.94885852e-16 -1.00236000e-18]
 [-1.50011672e-16  8.02068255e-17 -1.00000000e+00 -6.73412112e-18]
 [-1.14711926e-16  1.00000000e+00 -6.39496469e-17  5.00000000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Is the following matrix should be I (4x4):
 [[ 1.00000000e+00 -1.94885852e-16 -1.09017582e-16  5.55111512e-17]
 [ 1.50011672e-16  1.00000000e+00 -1.89744855e-17  1.62213639e-17]


Failed to establish dbus connection

**Answer 4.** Proof attached as a pdf file.

## 2: ICP alignment

1. Write a function that takes two point clouds as input without known correspondences and perform the iterative closest point algorithm.
2. Perform the ICP alignment between the two bunnies and plot their individual coordinate frames as done in class.
3. Does ICP always give the correct alignment? Why or Why not?
4. What are other variants of ICP and why are they helpful (you can look at point to plane ICP)?

In [8]:
from sklearn.neighbors import KDTree

def getNearestNeighbors(pts1, pts2):
    # pts1 : 3 x N
    pts1_temp, pts2_temp = np.copy(pts1.T), np.copy(pts2.T)
    # pts1_temp : N x 3
    tree = KDTree(pts1_temp)
    dist, ind = tree.query(pts2_temp, k = 1)

    return [dist, ind]

In [9]:
def visualiseBunnyPts(ptCloud, ptCloud2, mesh_frame, mesh_frame_transform, saveResults = False, iteration = 0):

    pcd1 = o3d.geometry.PointCloud()
    pcd1.points = o3d.utility.Vector3dVector(ptCloud.T)
    o3d.io.write_point_cloud("bunnyOriginal.ply", pcd1)

    # Load saved point cloud and visualize it
    bunny = o3d.io.read_point_cloud("bunnyOriginal.ply")

    pcd2 = o3d.geometry.PointCloud()
    pcd2.points = o3d.utility.Vector3dVector(ptCloud2.T)
    o3d.io.write_point_cloud("bunnyTransformed.ply", pcd2)

    # Load saved point cloud and visualize it
    bunny_transform = o3d.io.read_point_cloud("bunnyTransformed.ply")

    # Color the bunny red
    bunny.paint_uniform_color([1,0,0])

    # Color the transformed bunny blue
    bunny_transform.paint_uniform_color([0, 0, 1])

    # Visualize all of them together
    o3d.visualization.draw_geometries([bunny, mesh_frame, bunny_transform, mesh_frame_transform])

    if saveResults:
        o3d.io.write_point_cloud("results/ICP_at_" + str(iteration) + "iteration" + ".pcd", [bunny_transform, bunny, mesh_frame, mesh_frame_transform])

In [10]:
def ICP(pts1_input, pts2_input, coordinateframe1_input, coordinateframe2_input, iterations, displayStep = 1, tolerance = 0.00000000001):

    # Copy the input first, because they'll get changed/lost otherwise
    pts1, pts2 = np.copy(pts1_input), np.copy(pts2_input)
    coordinateframe1, coordinateframe2 = copy.deepcopy(coordinateframe1_input), copy.deepcopy(coordinateframe2_input)
    error = absoluteError(pts1, pts2)
    T_cumulative = np.eye(4)
    
    # Visualize the initial positions
    print("Orignial Locations...\n")
    visualiseBunnyPts(pts1, pts2, coordinateframe1, coordinateframe2)

    for iteration in range(1, iterations + 1):

        if iteration != 1 and error < tolerance:
            print("ICP stopped at the iteration ", iteration - 1, " with the absolute error as", error,", here the centroid of the 2 bunnies are", np.mean(np.mean(pts1)), np.mean(np.mean(pts2)),"\n")
            return

        [distance, indices] = getNearestNeighbors(pts1, pts2)
        pts2_nearest = pts2[:, indices][:, :, 0]
        [T, R, t] = procrustes_alignment(pts1, pts2_nearest)

        T_cumulative = T_cumulative @ np.linalg.inv(T)
        pts2 = getTransformedPoints(pts2, R, t[:, 0])
        coordinateframe2 = coordinateframe2.transform(T)
        error = absoluteError(pts1, pts2)
        print("Error at iteration no", iteration," is equal to", error, "\n")

        if(iteration % displayStep == 0 or iteration == iterations or error < tolerance):
            print("Visualization at the ", iteration, "iteration...\n")
            visualiseBunnyPts(pts1, pts2, coordinateframe1, coordinateframe2)

    print("ICP could not approximate the transmortation within", iterations, " iterations, the absolute error stagnated around", error," and the centroid of the 2 bunnies are", np.mean(np.mean(pts1)), np.mean(np.mean(pts2)),"\n")

In [11]:
#Visulaize initial configuration:
bunnyPath = "bunny.ply"
bunny = o3d.io.read_point_cloud(bunnyPath)

#Get the position matrix
bunnyPts = np.asarray(bunny.points).T

#get the coordinate axes
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=.1,origin=[0,0,0])

#Get the transformation
T_12 = np.eye(4)
R_12 = getRotationMatrix(0, 0, np.pi/2)
t_12 = np.array([0, 0, 5])
T_12[:3, :3], T_12[:3, 3] = R_12, t_12

bunnyPts2 = getTransformedPoints(bunnyPts, R_12, t_12)
mesh_frame_transform = copy.deepcopy(mesh_frame).transform(T_12)

# Set number of iterations to run
iterations = 100
ICP(bunnyPts, bunnyPts2, mesh_frame, mesh_frame_transform, iterations, 10)

Orignial Locations...

Error at iteration no 1  is equal to 0.003079529818050385 

Error at iteration no 2  is equal to 0.002235678472367941 

Error at iteration no 3  is equal to 0.0019780486325649176 

Error at iteration no 4  is equal to 0.001760232476250233 

Error at iteration no 5  is equal to 0.0017402267266533292 

Error at iteration no 6  is equal to 0.0016078964995538344 

Error at iteration no 7  is equal to 0.001611999422207814 

Error at iteration no 8  is equal to 0.001506431073726959 

Error at iteration no 9  is equal to 0.0014938808735914715 

Error at iteration no 10  is equal to 0.001404398221847353 

Visualization at the  10 iteration...

Error at iteration no 11  is equal to 0.0013816500143179098 

Error at iteration no 12  is equal to 0.0013019217413744628 

Error at iteration no 13  is equal to 0.0012766138675339947 

Error at iteration no 14  is equal to 0.0012063004136090144 

Error at iteration no 15  is equal to 0.0011813060021104902 

Error at iteration no 1

*THEORETICAL ANSWERS*

**Answer 3**

In genral, ICP doesn't always give the coorect alignment because of the noisy depths associated with the Lidar calculations. Even after assuming ideal non-noisy position coordinates the ICP alignment may not always work. The ICP relies heavily on the orthogonal procrustus alignment which relies on the correspondences between the two set of points. If the corresponsences are correct, the proctrustus would give us the answer in just one iteration, however if its not the correct one, the absolute error wouldn't have gone to close to zero values. Here, the absolute error remains much larger than the tolerance, hence we run the algorithm once again and repeat the process. At every ICP iteration, we perform a nearest neighbor search. What that does is basically for each point int set_2 it tries to find its nearest neighbor in set_1, this way by assuming the correspondences we apply the procrutus alignment. Therefore if the correspondences are nerver matched properly we will never get the correct alignment. After trying out a few testcases, we have noticed that when the either the bunny's orientation or its location is changed by a significant amount, it becomes very difficult for the bunny to get aligned to the desired orientation  but its *position still gets aligned pretty well*. This misalignment in orientation arises due to a certain *local minima in the error value* which forces the procrustus alignment to not change it significantly because no small change in the rotation matrix can give a lower error value than the one already achieved and in order to reach the perfect alignment (error = 0), the alignment must first increase the error value and then decrease to the global minima.

**Answer 4**

In the normal ICP algorithm each point in one data set is paired with the closest point in the other data set to form correspondences. Then a point to point error metric is calculated in which the sum of the squared distance between points in each correspondence pair is minimized. The process is iterated until the error becomes smaller than a tolerance value or it stops changing. On the other hand, in *point to plane ICP*, the error metric minimizes the sum of the squared distance between a point and the tangent
plane at its correspondence point, the figure uploaded in the results folder can be referred for more clarity on the geometry. Although each iteration of the point to plane ICP algorithm is generally slower than the point to point version, it has been observed the *convergence rates are faster* in the former, which is why it is so widely used.

Genralized ICP is an algorthm which aims to combine the Iterative Closest Point and the point to plane ICP algorithms into a single probabilistic framework. This framework is then used to model the locally planar surface structure from both scans instead of just one scan as done in the point to plane algorithm. This can be realized as a plane to plane ICP. The new approach is shown to have outperformed both the standard ICP and the point to plane ICP approaches. In addition to being *more robust* to incorrect correspondences, outliers, and noisy measurement depths, it has demostrated an *improved performance* level and can also *incorporate probabilistic models* while maintaining the normal ICP's speed and simplicity.